In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [3]:
#imports
from hb.clusterApp.display import loading
from hb.clusterApp import checksum
from hb.clusterApp import selectType

from hb import clusterApp

import warnings

from urllib.parse import unquote

import IPython
from IPython.display import clear_output

from ipywidgets import widgets
from ipywidgets import HTML
from ipywidgets import Layout
from ipywidgets import VBox, HBox
from ipywidgets import Image, Checkbox

In [5]:
#get the url
url = unquote(jupyter_notebook_url)

#worsheet id
wks = url.split('worksheetId=')[1].split('&')[0]
loading()

#worbook id
wkb = url.split('workbookId=')[1].split('&')[0]

#fix formatting
wkb = wkb.replace('"', '')
wks = wks.replace('"', '')

#url for api
api_url = url.split('data-lab')[0]

#authentication
auth_token = spy.client.auth_token

#instantiate the App
app = clusterApp.App(wkb, wks, api_url, auth_token, quiet = True)
signals = app.signals.Name.values
clear_output()

### Styling ###
_layout = Layout(flex='1 1 auto', width='99%')
_widget_style = {'description_width': '150px'}

_style = '''
<style>
    .container { width:90% !important; }
    .bottom-border {border-bottom: 2px solid black ; }
    .fast-spin {
                  -webkit-animation: fa-spin 2.5s infinite linear;
                  animation: fa-spin 2.5s infinite linear;
                }
    .prompt_container {width:0px; min-width:0px; visibility: collapse}
    .prompt {width:0px; min-width:0px; visibility: collapse}
    .output_subarea {width:100%; max-width:100%}
    .jupyter-widgets-view {max-width:70%; min-width:40%}
</style>

'''

### misc. configuration ###
warnings.simplefilter(action='ignore')

### Display ###
IPython.display.display(
    IPython.display.HTML(_style)
)

#Seeq logo
logo = HTML(
    "<img src='https://www.seeq.com/sites/default/files/Seeq_Logo_PMS%20close%20crop_0.png' width=100px>"
)

_under_logo = HTML('<h3>Scatterplot Clustering</h3>')

### Buttons and Widgets ###
_working = widgets.Output()

xsignal = widgets.Dropdown(
    description="x Signal",
    options=signals,
    value=signals[0],
    layout=_layout,
    style=_widget_style                    
)

ysignal = widgets.Dropdown(
    description='y Signal',
    options=signals,
    value=signals[1],
    layout=_layout,
    style=_widget_style
)

minClusterSize = widgets.Text(
    description='Min Cluster Points',
    value='200',
    layout=_layout,
    style=_widget_style
)

percentOfData = widgets.Text(
    description='Percent of Data (%)',
    value='5',
    layout=_layout,
    style=_widget_style
)

clusterExtent = widgets.Text(
    descripction='Cluster Extent',
    value='1',
    layout=_layout,
    style=_widget_style
)

exactBox = Checkbox(False, description='Ignore Percent of Data?')

buttonClusterUnsupervised = widgets.Button(description="Cluster", layout=_layout)
buttonSelectSupervised = widgets.Button(description="Visual (Supervised)", layout=_layout)
buttonSelectUnsupervised = widgets.Button(description="Density (Unsupervised)", layout=_layout)
buttonStartSupervised = widgets.Button(description="Start Selection", layout=_layout)
buttonClusterSupervised = widgets.Button(description="Define Cluster", layout=_layout)

#Choice of Supervised or Unsupervised:
typeSelection = [
    buttonSelectSupervised,
    buttonSelectUnsupervised
]

widgetDisplayUnsupervised = [
    HTML("""<div>
    <h4>Cluster Definition:</h4>
         <p>Minimum number of points to classify a cluster. Use Larger of:</p>
         </div>"""),
    percentOfData,
    minClusterSize,
    exactBox,
    HTML("""<div><h4>Cluster Extent:</h4>
         <p>Adjust cluster boundary length<p/></div>
         """),
    clusterExtent,
    buttonClusterUnsupervised
]

widgetDisplaySupervised = [
    xsignal, 
    ysignal, 
    buttonStartSupervised
]

### Clicking Functions ###

def selectUnsupervised(*args):
    selectType(widgetDisplayUnsupervised, buttonSelectSupervised, buttonSelectUnsupervised,)
    return

def selectSupervised(*args):
    selectType(widgetDisplaySupervised, buttonSelectSupervised, buttonSelectUnsupervised,)
    return

def uiClusterUnsupervised(*args):
    #will be closed
    buttons = [buttonSelectSupervised, buttonSelectUnsupervised, buttonClusterUnsupervised]
    clusterApp.clusterUnsupervised(app, buttons, signals, minClusterSize, exactBox, percentOfData, clusterExtent)
    return
def uiStartSupervised(*args):
    buttons = [buttonSelectSupervised, buttonSelectUnsupervised, buttonStartSupervised, xsignal, ysignal]
    
    global hist_grid_points
    global datadf
    
    datadf, hist_grid_points = clusterApp.startSupervised(app, buttons, xsignal, ysignal, buttonClusterSupervised)
    return

def uiClusterSupervised(*args):
    buttons = [buttonClusterSupervised]
    try:
        test = indexofselection
    except NameError:
        print('SELECTION ERROR. Please make a selection with the lasso tool first.')
        return
    clusterApp.clusterSupervised(app, buttons, xsignal, ysignal, clusterExtent, 
                                 datadf, indexofselection, hist_grid_points)
    return

buttonSelectSupervised.on_click(selectSupervised)
buttonSelectUnsupervised.on_click(selectUnsupervised)
buttonClusterUnsupervised.on_click(uiClusterUnsupervised)
buttonStartSupervised.on_click(uiStartSupervised)
buttonClusterSupervised.on_click(uiClusterSupervised)

display(VBox([HBox([logo,_working]),HBox([_under_logo])]), HBox(typeSelection))

IndexError: list index out of range